In [2]:
import xlrd

def Find_Distance(e, L) : #Function to retrieve Distance inside the GraphDist graph according to specific country e.
    
    for i in range(len(L)) :
        
        if L[i][0] == e :
            
            return L[i][1]
        
    return -1



def Find_Origin(e, L) : #Function to retrieve Origin inside the GraphDist graph according to specific distance e.
    
    for i in range(len(L)) :
        
        if L[i][1] == e :
            
            return L[i][0]
        
    return -1
            
    

def Find_Ingredient(d, L) : #Finds an ingredient according to destination.
    
    ingredient_found = False
    i = 0
    
    while(not(ingredient_found and i<len(L))) :
        
        if (L[i][3] == d) :
            
            ingredient_found = True
        
        i += 1
        
    if (i==len(L)) :
        
        if (L[i][3] == d) :
            
            return L[i][0]
        
        else :
            
            return "Unkown Destination"
    
    return L[i][0]


def Find_Min_CF_O_Ingredient(id, O, L) : #Finds for a determined Id and a specific origin a product with the lowest CF.
    
    ingredient_found = False
    i = 0
    min = 9999 #fictitious min for our CF. Normally CFs will not be that high for an ingredient.
    k = 0 #This will be the index of the minimal CF ingredient.
    
    while(not(ingredient_found)) :
        
        if (L[i][0] == id) :
            
            for j in range(3) : # '3' is equivalent to the 'gap' for the generalized version.
                
                if ((L[(i//3)*3 + j][2] < min) and (L[(i//3)*3 + j][3] == O)) :
                    
                    min = L[i//3 + j][2]
                    k = j
                    
            ingredient_found = True
            
        i +=1
    
    return L[k][0]

def Find_Min_CF_Ingredient(id, L) : #Finds for a determined Id a product with the lowest CF.
    
    ingredient_found = False
    i = 0
    min = 9999 #fictitious min for our CF. Normally CFs will not be that high for an ingredient.
    k = 0 #This will be the index of the minimal CF ingredient.
    
    while(not(ingredient_found)) :
        
        if (L[i][0] == id) :
            
            for j in range(3) : # '3' is equivalent to the number of kinds of an ingredient (to be changed for the generalized version).
                
                if (L[(i//3)*3 + j][2] < min) :
                    
                    min = L[(i//3)*3 + j][2]
                    k = (i//3)*3 + j
                    
            ingredient_found = True
            
        i +=1
    
    return L[k][0]


def Find_Max_CF_Ingredient(id, L) : #Finds for a determined Id a product with the highest CF.
    
    ingredient_found = False
    i = 0
    max = 0 #fictitious max for our CF. Normally CFs will not be that high for an ingredient.
    k = 0 #This will be the index of the minimal CF ingredient.
    
    while(not(ingredient_found)) :
        
        if (L[i][0] == id) :
            
            for j in range(3) : # '3' is equivalent to the 'gap' for the generalized version.
                
                if (L[(i//3)*3 + j][2] > max) :
                    
                    max = L[(i//3)*3 + j][2]
                    k = (i//3)*3 + j
                    
            ingredient_found = True
            
        i +=1
    
    return L[k][0]

def Retrieve_Better_Ingredient(e,GraphCN) :
    
    CCF = GraphCN[e][2] #Current CF
    idx = (e//3)*3 #The index where our current ingredient type starts
    betterI = e #Index of the better ingredient.
    
    #We want to find THE better ingredient (i.e. the one that has the closest yet lower CF)
    
    gapMin = 99999 #Gap between two CFs

    #Let's thus find the minimal gap.
    
    for i in range (3) : 
            
        gap = CCF - GraphCN[idx+i][2] 
            
        if ((gapMin > gap) and (gap>0)) :
                
            gapMin = gap
            betterI = idx+i
    
    return betterI

def Calculate_the_RMA(GraphCN, nb_Ingredients, Vol) :
    
    RMA = 0
    
    for i in range(nb_Ingredients) :
        
        min = Find_Min_CF_Ingredient(i*3, GraphCN)
        max = Find_Max_CF_Ingredient(i*3, GraphCN)
        
        RMA += Vol[i]*((GraphCN[min][2]+GraphCN[max][2])/2)
        
    return RMA
        
        
        
        
        
    

In [3]:
#We suppose that 1L = 1kg (water at its maximum density i.e. 4°C)

Conversion_Matrix = [["0","kg","g","mg","cuillère à café","cuillère à soupe","L","dL","cL","mL"],
                    ["kg","1","1000","1000000","200","66.7","1","10","100","1000"],
                    ["g","0.001","1","1000","0.2","0.067","0","0","0","0"],
                    ["mg","0.000001","0.001","1","0.0002","0.000067","0","0","0","0"],
                    ["cuillère à café","0.005","5","5000","1","0.3","0","0","0","0"],
                    ["cuillère à soupe","0.015","15","15000","3","1","0","0","0","0"],
                    ["l","1","1000","1000000","200","66.7","1","10","100","1000"],
                    ["dl","0.1","100","100000","20","6.67","0.1","1","10","100"],
                    ["cl","0.01","10","10000","2","0.667","0.01","0.1","1","10"],
                    ["ml","0.001","1","1000","0.2","0.067","0.001","0.01","0.1","1"]]

In [4]:
#Simulation Database
#Price and Origin Graph (prices are in usd) :
#Reminder : 1kg of coffe ≃ 28.38L produced. 

recipeName ="Crème de carotte au café, chips de betterave."


GraphPO = [[0, "carotte",1.42,"Finlande","conventionnel"], 
           [1, "carotte",2.48,"Finlande","organique"], 
           [2, "carotte",1.32,"Italie","conventionnel"],
           [3, "crème liquide",2.96, "France","conventionnel"],
           [4, "crème liquide",2.90, "USA","conventionnel"],
           [5, "crème liquide", 5.65, "Argentine","conventionnel"],
           [6, "baies roses", 32.02, "France", "conventionnel"],
           [7, "baies roses",53.44, "Brésil","conventionnel"],
           [8, "baies roses", 29.01,"RU", "conventionnel"],
           [9, "café", 12.31,"France","conventionnel"],
           [10, "café",25.56, "RU", "conventionnel"],
           [11, "café",25.85, "USA","conventionnel"],
           [12, "fleur de sel", 23.99, "France","conventionnel"],
           [13, "fleur de sel", 19.99, "Canada","conventionnel"],
           [14, "fleur de sel",45.00, "Cambodge","conventionnel"],
           [15, "betterave", 1.77, "France","conventionnel"],
           [16, "betterave",0.60, "Inde","conventionnel"],
           [17, "betterave",2.50, "RU","conventionnel"],
           [18, "Huile d’olive", 6.33,"France","conventionnel"],
           [19, "Huile d’olive", 10.5,"USA","conventionnel"],
           [20, "Huile d’olive",12.98, "Japon", "conventionnel"]]
    
    #CF and Nutrition Factors Graph (CF is in CO2(KG) emmited per KG of production) :
    #The final CF will be calculated thanks to the origin of the product data.
    #For the ingredients that don't have a CF we take an average value for all the same products from all destinations.
    #For the sour cream (crème liquide), we said it was the same as the dairy products.
    #For the berries : we take lowest and highest of fruits/vegetables from a specific website.
    #What about the error rate?
    #For the ingredients that not posses a CF from a certain destinatnion we take the average of all the other know 
    #destinations.
    #Average has been done for : pink berries, beet, olive oil.


GraphCN = [[0, 'carrote', 0.142, 'Finlande', 'conventionnel'], 
          [1, 'carotte', 0.003, 'Finlande', 'conventionnel'], 
          [2, 'carotte', 0.283, 'Italie', 'conventionnel'], 
          [3, 'crème liquide', 1.38, 'France', 'conventionnel'], 
          [4, 'crème liquide', 3.34, 'Argentine', 'conventionnel'], 
          [5, 'crème liquide', 1.15, 'USA', 'conventionnel'], 
          [6, 'baies roses', 1.9, 'France', 'conventionnel'], 
          [7, 'baies roses', 1.9, 'Brésil', 'conventionnel'], 
          [8, 'baies roses', 1.9, 'RU', 'conventionnel'], 
          [9, 'café', 0.5, 'USA', 'conventionnel'], 
          [10, 'café', 0.08, 'RU', 'conventionnel'], 
          [11, 'café', 0.07, 'France', 'conventionnel'], 
          [12, 'fleur de sel', 4.1, 'France', 'conventionnel'], 
          [13, 'fleur de sel', 4.1, 'Canada', 'conventionnel'], 
          [14, 'fleur de sel', 4.1, 'Cambodge', 'conventionnel'], 
          [15, 'betterave', 1.9, 'France', 'conventionnel'], 
          [16, 'betterave', 1.9, 'Inde', 'conventionnel'], 
          [17, 'betterave', 1.9, 'RU', 'conventionnel'], 
          [18, 'Huile d’olive', 10.51, 'France', 'conventionnel'], 
          [19, 'Huile d’olive', 6.4, 'USA', 'conventionnel'], 
          [20, 'Huile d’olive', 8.46, 'Japon', 'conventionnel']]
    

    #Distances Graph (in km and from Japan). We suppose that our cooker is located in Japan, Tokyo. 
    #Distances are calculated from capital to capital.

GraphDist = [["Finlande",7820],
            ["Italie",9857],
            ["France",9714],
            ["Argentine",18365],
            ["USA",10903],
            ["Brésil",17673],
            ["RU",9561],
            ["Canada",10319],
            ["Cambodge",4409],
            ["Inde",5840],
            ["Japon",0]]

In [5]:
def Information_Retrieval(GraphPO, GraphCN, GraphDist) :

    #Retrieval of our recipe in a python array
    
    workbook = xlrd.open_workbook('./Damien data test/R2018010124_Crème de carotte au café, chips de betterave_fr.xlsx')
    worksheet = workbook.sheet_by_index(0)

    V = [0]*2
    V[0] = [worksheet.cell(0, i) for i in range(worksheet.ncols)]
    V[1] = [worksheet.cell(1, i) for i in range(worksheet.ncols)]
    
    I = [] #Ingredients array
    Vol = [] #Volumes array
    Met = [] #Metrics array
    
    for e in V[0]:
    
        #We verify that we actually have an ingredient here :
        if (e.value=="ingredient" and V[1][V[0].index(e)].value!=""):
        
            if (V[1][V[0].index(e)].value=="Huile d’olive "):
                
                if (V[1][V[0].index(e)+3].value==""):
                
                    Vol.append(22.5)
                
                    Met.append("ml")
                
        
            I.append(V[1][V[0].index(e)].value) #We add our new ingredient
        
        
        elif (e.value=="volume" and V[1][V[0].index(e)].value!=""):
        
            if (V[1][V[0].index(e)+1].value=="unité"):#We will normally ask our KB here instead of mannualy searching 
                                                      #weight of an unit on the net.        
            
                Vol.append(300 * V[1][V[0].index(e)].value)
            
            else: 
    
                Vol.append(V[1][V[0].index(e)].value) #We add our new volume
        
        elif (e.value=="metric" and V[1][V[0].index(e)].value!="" and V[1][V[0].index(e)].value!="unité"):
        
            Met.append(V[1][V[0].index(e)].value) #We add our new volume
            
        elif (e.value=="metric" and V[1][V[0].index(e)].value!="" and V[1][V[0].index(e)].value=="unité"):
        
            Met.append("g") #We add our new volume
            
    I = [i.strip() for i in I]
    Met = [m.strip() for m in Met]
    nb_Ingredients = len(I)
    
    #Converting our Volumes, this time with the Conversion Matrix :

    Vol2 = []
    Met2 = []

    for v in Vol:

        idx = Vol.index(v)
        
        l = 0
        n = len(Conversion_Matrix[0])
        
        while ((l<n) and (Met[idx]!=Conversion_Matrix[l][0])) :
            
            l += 1
            
        if (l == n) : #Dans le cas ou la métrique n'est pas trouvable dans la matrice.
                
            print("The metric "+Met[idx]+" cannot be found in the Conversion Matrix.")
        
        else :
            
            Vol2.append(v*float(Conversion_Matrix[l][1]))
            
    
    #We now add information about the CO2 produced by traveling that distance :
    
    # 1 km = 1 kcal of fossil fuel. 
    # 1 gallon of gasoline = 31 kcal = 8.8 kg of CO2
    
    for i in range(len(GraphDist)) :
        
        GraphDist[i].append((GraphDist[i][1]/31)*8.8)
    
    #We calculate the prices and CFs according to our weights (in KG and L) :
    
    I = [0]*nb_Ingredients
    P = [0]*nb_Ingredients
    C = [0]*nb_Ingredients
    
    #################### CHOICE OF INGREDIENTS - PART I ####################
    
    #We choose by default the ingredient in our recipe that is closest to the cooker.
    
    i = 0
    
    while i < (len(GraphPO) - 2) :
        
        min = 20027  #The closest country to the cooker's location. Initialised with biggest distance between two 
                     #places in the world (km).
            
        j = 0
        k = 0 #The index of that ingredient
        
        minCF = False #This boolean will be used in the case we have two same destinations for a same ingredients. 
        
        while j < 3 : #In our database we always have three different options per ingredient. 
                      #In the general case we will have to retrieve the "gap" between current and next ingredient 
                      #type.
        
            O = GraphPO[i+j][3]
            Dist = Find_Distance(O,GraphDist)

            if (min > Dist) :
                
                min = Dist
                minCF = False
                k = j

            elif (min == Dist) : #In the case we have default ingredients with same origin : we will choose 
                                 #according to the lowest CF.
                    
                I[i//3] = [GraphPO[i+j][0], Find_Origin(min, GraphDist)]  #We save the ID of our ingredient.
                minCF = True
            
            j += 1
        
        if (not(minCF)) :
        
            I[i//3] = i+k 
                
        i += 3
        
    #################### CHOICE OF INGREDIENTS - PART II ####################
     
    #We will now replace the origin destinations with our ingredients.    
        
    #We will also replace the ingredients that have an ID as an element with the ingredient that has the lowest CF
    #for that common ingredient type (example : Conventional Carrots and Organic Carrots from Finland). 
           
        
    for i in range(nb_Ingredients) : 
        
        if (not(isinstance(I[i],int))) :
            
            I[i] = Find_Min_CF_O_Ingredient(I[i][0],I[i][1],GraphCN)
    
    #We now have the list of our ingredients
    #We then calculate the corresponding Prices and CFs according to our ingredients and their weights for our Recipe.
    
    #Calculating Prices and CFs
    
    print(Vol2)
    
    for i in range(nb_Ingredients) :
        
            P[i] = Vol2[i] * GraphPO[I[i]][2]
            C[i] = format(Vol2[i] * GraphCN[I[i]][2],'.5f') #Otherwise results are too long.
            C[i] = float(C[i])
    
    print("I = "+ str(I))
    print("P = "+ str(P))
    print("C = "+ str(C))
    
    return I, P, C, Vol2

In [6]:
Information_Retrieval(GraphPO, GraphCN, GraphDist)


[0.8, 0.2, 0.005, 0.1, 0.005, 0.6, 0.0225]
I = [1, 3, 8, 10, 14, 16, 20]
P = [1.984, 0.592, 0.14505, 2.556, 0.225, 0.36, 0.29205]
C = [0.0024, 0.276, 0.0095, 0.008, 0.0205, 1.14, 0.19035]


([1, 3, 8, 10, 14, 16, 20],
 [1.984, 0.592, 0.14505, 2.556, 0.225, 0.36, 0.29205],
 [0.0024, 0.276, 0.0095, 0.008, 0.0205, 1.14, 0.19035],
 [0.8, 0.2, 0.005, 0.1, 0.005, 0.6, 0.0225])

In [7]:
def Recipe_Evaluation(I, P, C, Money_threshold, DCF) :
    
    SCF = 0
    SP = 0
    nb_Ingredients = len(I)
    
    for i in range(nb_Ingredients) :
        
        SCF = SCF + C[i]
        SP = SP + P[i]
        
    if DCF >= SCF : 
        
        print("")
        print("Your recipe meets the CF requirements :")
        print("Desired CF = "+str(DCF) + " kgCO2")
        print("Recipe's CF = "+str(SCF)+ " kgCO2")

    else :
        
        print("")
        print("Your recipe needs to be updated to meet your requirements.")
        print("")
        print("Current CF of your recipe is : "+str(SCF)+ " kgCO2")
        print("Current Price of your recipe is : "+ str(format(SP,'.2f')) + " USD") 
        Recipe_Update_Algorithm(I, P, C, SCF, SP, Money_Threshold, DCF, GraphPO, GraphCN, Vol)
    

    
    
    

In [8]:
def Recipe_Update_Algorithm(I, P, C, SCF, SP, Money_Threshold, DCF, GraphPO, GraphCN, Vol) :
    
    Same_Recipe = False
    i = 0
    Old_Recipe = I
    New_Recipe = I
    nb_ingredients = len(I)
    
    while (DCF < SCF and not(Same_Recipe)) :
        
        BetterI = Retrieve_Better_Ingredient(I[i],GraphCN)
        PBetter = Vol[i] * GraphPO[BetterI][2]
        
        if (SP - P[i] + PBetter) < Money_Threshold : 
            
            New_Recipe[i] = BetterI
            CFBetter = Vol[i] * GraphCN[BetterI][2]
            SP = SP - P[i] + PBetter
            SCF = SCF - C[i] + CFBetter
            
        i += 1
        
        if (i == nb_ingredients) :
            
            i = 0
            
            if (New_Recipe == Old_Recipe) :
                
                Same_Recipe = True
                
            else : 
                
                Old_Recipe = New_Recipe
                
    print("")            
    print("Your Update is Done.")            
    print("New Recipe is : " + str(New_Recipe))
    print("New CF is : " + str(format(SCF,'.5f')) + " kgCO2")
    print("New Price of your Recipe is : " + str(format(SP,'.2f')) + " USD")
    
    return New_Recipe, SCF
            
            
                
        

In [9]:
Money_Threshold = 7 #7 USD

(I,P,C,Vol) = Information_Retrieval(GraphPO, GraphCN, GraphDist)
nb_Ingredients = len(I)
CF = 0
for i in range(nb_Ingredients) :
    CF += C[i]
RMA = Calculate_the_RMA(GraphCN,nb_Ingredients,Vol)
print("")
print("Your Input recipe is : "+recipeName)
print("")
print("The current Carbon Footprint of your recipe is : "+ str(CF) + " kgCO2 while the average Carbon Footprint for this recipe is : "+ str(RMA)+ " kgCO2.")
print("")
DCF = input("Please insert your Desired Carbon Footprint : ") 
DCF = float(DCF)
print("")
print("You entered DCF = "+str(DCF) +" kgCO2")
Recipe_Evaluation(I, P, C, Money_Threshold, DCF)

[0.8, 0.2, 0.005, 0.1, 0.005, 0.6, 0.0225]
I = [1, 3, 8, 10, 14, 16, 20]
P = [1.984, 0.592, 0.14505, 2.556, 0.225, 0.36, 0.29205]
C = [0.0024, 0.276, 0.0095, 0.008, 0.0205, 1.14, 0.19035]

Your Input recipe is : Crème de carotte au café, chips de betterave.

The current Carbon Footprint of your recipe is : 1.64675 kgCO2 while the average Carbon Footprint for this recipe is : 1.9521374999999999 kgCO2.

Please insert your Desired Carbon Footprint : 1.5

You entered DCF = 1.5 kgCO2

Your recipe needs to be updated to meet your requirements.

Current CF of your recipe is : 1.64675 kgCO2
Current Price of your recipe is : 6.15 USD

Your Update is Done.
New Recipe is : [1, 5, 8, 11, 14, 16, 19]
New CF is : 1.55340 kgCO2
New Price of your Recipe is : 6.67 USD
